In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# https://scikit-learn.org/stable/auto_examples/decomposition/plot_incremental_pca.html#sphx-glr-auto-examples-decomposition-plot-incremental-pca-py
# Authors: Kyle Kastner
# License: BSD 3 clause

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris
from sklearn.decomposition import PCA, IncrementalPCA

iris = load_iris()
X = iris.data
y = iris.target

n_components = 2
ipca = IncrementalPCA(n_components=n_components, batch_size=10)
X_ipca = ipca.fit_transform(X)

pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(X)

colors = ["navy", "turquoise", "darkorange"]

for X_transformed, title in [(X_ipca, "Incremental PCA"), (X_pca, "PCA")]:
    plt.figure(figsize=(8, 8))
    for color, i, target_name in zip(colors, [0, 1, 2], iris.target_names):
        plt.scatter(
            X_transformed[y == i, 0],
            X_transformed[y == i, 1],
            color=color,
            lw=2,
            label=target_name,
        )

    if "Incremental" in title:
        err = np.abs(np.abs(X_pca) - np.abs(X_ipca)).mean()
        plt.title(title + " of iris dataset\nMean absolute unsigned error %.6f" % err)
    else:
        plt.title(title + " of iris dataset")
    plt.legend(loc="best", shadow=False, scatterpoints=1)
    plt.axis([-4, 4, -1.5, 1.5])

plt.show()

In [ ]:
X_transformed.shape

In [ ]:
import sys
sys.path.append("../..")  # as a hack to import
#from rpca.pcp import pcp
from rpca.mwrpca import mwrpca
from rpca.stoc_rpca  import stoc_rpca
from rpca.omwrpca  import omwrpca
from rpca.omwrpca_cp  import omwrpca_cp


In [ ]:
# lambda_12 = lambda1 = 1.0/np.sqrt(max(X.shape))
# Lhat, Shat, rank, Uhat = stoc_rpca(X, lambda1 = lambda_12, lambda2 = lambda_12, burnin=10)

from rpca.pcp import pcp_v
L, S, niter, rank, V = pcp_v(X)

In [ ]:
import seaborn as sb

fig, axs = plt.subplots(1, 2)
sb.heatmap(pca.components_, ax=axs[0])
axs[0].set_title('components')
sb.heatmap(V[0:rank,:], ax=axs[1])
axs[1].set_title('V')

In [ ]:
X_pcp_trans = np.dot(X, V[0:rank,:].T)
print(X_pcp_trans.shape)

for color, i, target_name in zip(colors, [0, 1, 2], iris.target_names):
    plt.scatter(
        X_pcp_trans[y == i, 0],
        X_pcp_trans[y == i, 1],
        color=color,
        lw=2,
        label=target_name,
    )
# seperation is still very good

In [ ]:
sb.pairplot(pd.DataFrame(X_pcp_trans))
np.corrcoef(X_pcp_trans.T) # does not decorrelate

In [ ]:
# Classic PCA
sb.pairplot(pd.DataFrame(X_transformed))
np.corrcoef(X_transformed.T)  # does decorrelate

In [ ]:
X_transformed_ = np.dot(X-X.mean(), pca.components_.T)
X_transformed - X_transformed_

In [ ]:
# Principal Component Pursuit reconstruction

import seaborn as sb

fig, axs = plt.subplots(1, 3)
sb.heatmap(X, ax=axs[0])
axs[0].set_title('X')
sb.heatmap(L, ax=axs[1])
axs[1].set_title('L')
sb.heatmap(S, ax=axs[2])
axs[2].set_title('S')

In [ ]:
# PCA reconstruction

Lhat = pca.inverse_transform(X_pca)
Shat = Lhat - X

fig, axs = plt.subplots(1, 3)
sb.heatmap(X, ax=axs[0])
axs[0].set_title('X')
sb.heatmap(Lhat, ax=axs[1])
axs[1].set_title('Lhat')
sb.heatmap(Shat, ax=axs[2])
axs[2].set_title('Shat')


In [ ]:
for color, i, target_name in zip(colors, [0, 1, 2], iris.target_names):
    plt.scatter(
        X[y == i, 0],
        X[y == i, 1],
        color=color,
        lw=2,
        label=target_name,
    )